<a href="https://colab.research.google.com/github/rpatel71/Design-Optimization/blob/main/Homework_5/hw5_sqp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing all the libraries
import numpy as np
from matplotlib import pyplot as plt
import torch as t

In [ ]:
# defining the objective function as well as inequality constraints
function = lambda x: x[0]**2 + (x[1] - 3)**2
const_1 = lambda x: x[1]**2 - 2*x[0]
const_2 = lambda x: (x[1]-1)**2 + 5*x[0]-15

grad_function = lambda x : torch.tensor([[2 * x[0], 2 * (x[1] - 3)]])
grad_const_1 = lambda x : torch.tensor([[-2, 2 * x[1]]])
grad_const_2= lambda x: torch.tensor([[5, 2 * (x[1] - 1)]])

g = t.tensor[const_1, const_2]
grad_g = t.tensor[grad_const_1, grad_const_2]